In [31]:
import pandas as pd

In [32]:
with open('../datasets/cve_info.csv', 'r') as f:
    df_cve_nvd = pd.read_csv(f)
df_cve_nvd['published'] = pd.to_datetime(df_cve_nvd['published'])
df_cve_first = pd.read_pickle('../datasets/dataset.pkl')
cve_nvd = set(df_cve_nvd['id'].unique())
cve_first = set(df_cve_first['cve'].unique())

In [38]:
print(f"#CVEs from NVD:   {len(cve_nvd)}")
print(f"#CVEs from FIRST: {len(cve_first)}")

#CVEs from NVD:   241103
#CVEs from FIRST: 239192


In [34]:
missing_first = cve_nvd - cve_first
missing_nvd = cve_first - cve_nvd
print(f"Missing from FIRST dataset wrt NVD: {len(missing_first)}")
print(f"Missing from NVD dataset wrt FIRST: {len(missing_nvd)}")

Missing from FIRST dataset wrt NVD: 1960
Missing from NVD dataset wrt FIRST: 49


In [35]:
count_rejected = 0
count_received = 0
missing = []
for cve in missing_first:
    data = df_cve_nvd[df_cve_nvd['id'] == cve]['vulnStatus'].values
    if data == 'Rejected':
        count_rejected += 1
    elif data == 'Received':
        count_received += 1
    else:
        missing.append(cve)
print(f"Missing in FIRST dataset: {missing}")
print(f"Rejected: {count_rejected}")
print(f"Received: {count_received}")
print(f"Total: {count_rejected + count_received}/{len(missing_first)}")

Missing in FIRST dataset: ['CVE-2005-2759']
Rejected: 1819
Received: 140
Total: 1959/1960


In [58]:
vuln_status = list(df_cve_nvd['vulnStatus'].unique())
print('NVD')
for status in sorted(vuln_status):
    print(f"{status}: {len(df_cve_nvd[df_cve_nvd['vulnStatus'] == status])}")
print('\nFIRST')
counts = df_cve_nvd[df_cve_nvd['id'].isin(cve_first)]['vulnStatus'].value_counts()
for status in sorted(vuln_status):
    print(f"{status}: {counts[status]}")

NVD
Analyzed: 130652
Awaiting Analysis: 2521
Modified: 93779
Received: 170
Rejected: 13970
Undergoing Analysis: 11

FIRST
Analyzed: 130651
Awaiting Analysis: 2521
Modified: 93779
Received: 30
Rejected: 12151
Undergoing Analysis: 11
